In [ ]:
# Install required libraries
!pip install deepgram-sdk==2.12.0 openai python-docx

# Import necessary libraries
import os
import json
from deepgram import Deepgram
import openai
from openai import OpenAI
from docx import Document
from datetime import datetime
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.2/149.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
# 1. Upload your audio file
print("Please upload your audio file:")
uploaded = files.upload()
audio_path = next(iter(uploaded.keys()))

Please upload your audio file:


Saving ultrasound.ogg to ultrasound.ogg


In [ ]:
# 2. Set API keys
DEEPGRAM_API_KEY = "your key"  # Replace with your actual key

deepgram = Deepgram(DEEPGRAM_API_KEY)

client = OpenAI(api_key="your key")


In [ ]:
# 3. Transcribe with Deepgram Nova-3 Medical
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

async def transcribe_audio():
    with open(audio_path, 'rb') as audio:
        source = {'buffer': audio, 'mimetype': 'audio/wav'}
        options = {
            'model': 'nova-3-medical',  # Use the medical specialized model
            'language': 'en',
            'smart_format': True,
            'diarize': True
        }

        response = await deepgram.transcription.prerecorded(source, options)
        # The response is already a dictionary, so access the data directly
        return response['results']['channels'][0]['alternatives'][0]['transcript']

async def main():
  transcript = await transcribe_audio()
  print("Transcription complete")
  print(f"Length of transcript: {len(transcript)} characters")
  return transcript

transcript = asyncio.run(main()) # Execute using asyncio.run

Transcription complete
Length of transcript: 830 characters


In [ ]:
transcript

"Liver size, point nine centimeters, moderate hepatomegaly with grade two fatty change in pressure. G b distended wall, normolar. RK, 11.6 into 4.1 centimeters. Pancreas in normal manner. Paratic area obscured manner. Bladder distended. Prostate 20 cc. Calculus in calyxin measuring Right kidney. Calculus in index, callus measuring. Four and, ma'am. Note on the GTO. Spleen, 9.5. L k, 10.5, four point six. Umbilical hernia through defective measuring. 1.5 centimeters with herniation of omentum. Impression, umbilical hernia, nonobstructive, right renal calculus, Four calculus. No evidence of right inguinal hernia at present. And no EVT. No evidence of all related mass per collection and thickening in relation to visualized loop of the loops in ROV. Appendix notch separately seen. No. No EVM. Appendix notch separately seen."

In [ ]:
# 4. Process with GPT-4 to extract structured information
system_prompt = """You are a medical document specialist. Extract detailed information from this ultrasound transcription to complete a structured USG Abdomen report.
Format your response as a structured JSON with the following fields:
{
  "patient_id": "",
  "patient_name": "",
  "age_gender": "",
  "liver": "",
  "gallbladder": "",
  "spleen": "",
  "pancreas": "",
  "aorta": "",
  "right_kidney": "",
  "left_kidney": "",
  "bladder": "",
  "prostate": "",
  "rif": "",
  "other_findings": "",
  "impression": ""
}
Include all measurements, observations, and medical terminology exactly as described in the transcript. If any field is not mentioned, leave it blank.
Format the impression as a comma-separated list of key abnormal findings."""

# Use openai.ChatCompletion.create() for chat completions, but remove response_format
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": transcript}
    ],
    # response_format={"type": "json_object"}  # Remove this line - not supported by GPT-4 chat completions
)

# Access the content of the message and parse as JSON
medical_data = json.loads(response.choices[0].message.content)

print("Medical information extracted successfully")

Medical information extracted successfully


In [ ]:
# 5. Generate USG Abdomen report using the template
doc = Document()

# Add title
doc.add_heading('USG ABDOMEN', 0)

# Add current date in the required format (DD-MM-YYYY)
current_date = datetime.now().strftime("%d-%m-%Y")
date_paragraph = doc.add_paragraph()
date_paragraph.add_run(f"Report Date: {current_date}").bold = True
date_paragraph.alignment = 2  # Right alignment

# Add patient information section
patient_info = doc.add_paragraph()
patient_info.add_run(f"Patient ID: {medical_data.get('patient_id', '_________')}    ")
patient_info.add_run(f"Name: {medical_data.get('patient_name', '_________')}    ")
patient_info.add_run(f"Age/Gender: {medical_data.get('age_gender', '_____')}")

# Add FINDINGS section
doc.add_heading('FINDINGS:', level=1)

# Add each organ section with proper formatting
for organ_name, field_name in [
    ('Liver:', 'liver'),
    ('Gallbladder:', 'gallbladder'),
    ('Spleen:', 'spleen'),
    ('Pancreas:', 'pancreas'),
    ('Aorta:', 'aorta')
]:
    p = doc.add_paragraph()
    p.add_run(organ_name).bold = True
    p.add_run(f" {medical_data.get(field_name, '')}")

# Add Kidneys with indented right and left
p = doc.add_paragraph()
p.add_run('Kidneys:').bold = True
doc.add_paragraph(f"    Right Kidney: {medical_data.get('right_kidney', '')}")
doc.add_paragraph(f"    Left Kidney: {medical_data.get('left_kidney', '')}")

# Add remaining sections
for organ_name, field_name in [
    ('Bladder:', 'bladder'),
    ('Prostate:', 'prostate'),
    ('Rif:', 'rif')
]:
    p = doc.add_paragraph()
    p.add_run(organ_name).bold = True
    p.add_run(f" {medical_data.get(field_name, '')}")

# Add other findings if available
if medical_data.get('other_findings'):
    p = doc.add_paragraph()
    p.add_run(medical_data.get('other_findings', ''))

# Add IMPRESSION section
doc.add_heading('IMPRESSION:', level=1)
impression = doc.add_paragraph(medical_data.get('impression', ''))

# Save the completed report
output_filename = f"USG_ABDOMEN_Report_{current_date.replace('-','_')}.docx"
doc.save(output_filename)
print(f"Report generated successfully as: {output_filename}")

# Download the generated file
files.download(output_filename)

Report generated successfully as: USG_ABDOMEN_Report_22_03_2025.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>